**DATA CLEANING / NORMALIZATIO :
1) Noise Reduction **

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install librosa noisereduce
!pip install pydub

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import noisereduce
from google.colab import files
import io
import soundfile as sf
import numpy as np
import shutil

In [ ]:
path_s = '/content/drive/MyDrive/RAW_DATA'
file_s = os.listdir(path_s)

In [ ]:
for j in file_s:
  # path=os.path.join(path_s,j)
  # files=os.listdir(path)
  # for i in files:
    os.makedirs('/content/uploaded_audio')
    os.makedirs('/content/normalized_audio')
    os.makedirs('/content/denoised_audio')
    os.makedirs('/content/resampled_audio')
    os.makedirs('/content/clipped_audio')

    up_audio_path= os.path.join(path_s,j)
    up_audio_files = os.listdir(up_audio_path)

  # NOISE REDUCTION
    nor_audio_path = up_audio_path
    nor_audio_files = up_audio_files

    for nor_audio_file in nor_audio_files:
      if('.flac' in nor_audio_file):
        nor_audio_filepath = os.path.join(nor_audio_path, nor_audio_file)
        nor_audio_data, sr = librosa.load(nor_audio_filepath, sr=None)
        reduced_noise = noisereduce.reduce_noise(y=nor_audio_data, sr=sr)
        den_filename = f'{nor_audio_file}'
        den_filepath = os.path.join('/content/denoised_audio', den_filename)
        sf.write(den_filepath, reduced_noise, sr)

  #SAMPLE RATE ADJUSTMENT
    d_audio_path = '/content/denoised_audio'
    d_audio_files = os.listdir(d_audio_path)

    for d_audio_file in d_audio_files:
      if('.flac' in d_audio_file):
        d_audio_filepath = os.path.join(d_audio_path, d_audio_file)
        denoised_audio, den_sr = librosa.load(d_audio_filepath)
        target_sampling_rate = 16000
        resampled_audio = librosa.resample(denoised_audio, orig_sr=den_sr, target_sr=target_sampling_rate)
        resampled_filename = f'{d_audio_file}'
        resampled_filepath = os.path.join('/content/resampled_audio', resampled_filename)
        sf.write(resampled_filepath, resampled_audio, target_sampling_rate)

  # INTENSITY LEVEL FILTERING
    i_audio_path = '/content/resampled_audio'
    i_audio_files = os.listdir(i_audio_path)

    for i_audio_file in i_audio_files:
      if('.flac' in i_audio_file):
        i_audio_filepath = os.path.join(i_audio_path, i_audio_file)
        in_audio, sr = librosa.load(i_audio_filepath)
        y_trimmed, index = librosa.effects.trim(in_audio, top_db=20)
        clipped_dir = '/content/clipped_audio'
        os.makedirs(clipped_dir, exist_ok=True)
        clipped_filename = f'{i_audio_file}'
        clipped_filepath = os.path.join(clipped_dir, clipped_filename)
        sf.write(clipped_filepath, y_trimmed, sr)
        shutil.copy(clipped_filepath,"/content/drive/MyDrive/PROCESSED/CLEAN_DATA")

    %rm -rf clipped_audio
    %rm -rf uploaded_audio
    %rm -rf normalized_audio
    %rm -rf denoised_audio
    %rm -rf resampled_audio

In [ ]:
# print("DURATION",max(dur),"-",min(dur))
# print("AMP MAX",max(ampa),"-",min(ampa))
# print("AMP MIN",max(amp_na),"-",min(amp_na))


DURATION 8.103764172335602 - 0.09287981859410431
AMP MAX 1.0 - 0.3218994
AMP MIN 3.0517578e-05 - 0.0
